# Fourier-Transformation - Signale & Systeme

Martikelnummer: 7531990

In [1]:
import librosa
import os
import numpy as np
from cmath import sqrt
import ipywidgets as widgets
from ipywidgets import interact, interactive, IntSlider, FloatSlider, FloatLogSlider, RadioButtons
import IPython.display as ipd
import matplotlib.pyplot as plt

## Initialisierung

In [2]:
DIR = os.path.abspath('')
multi400 = os.path.join(DIR, "sounds\\410And420And430.wav")
multi456 = os.path.join(DIR, "sounds\\410AND510AND610.wav")
square100 = os.path.join(DIR, "sounds\\square-100Hz.wav")
lowFreq = os.path.join(DIR, "sounds\\lowFreq.wav")
highFreq = os.path.join(DIR, "sounds\\highFreq.wav")
highAmp = os.path.join(DIR, "sounds\\highAmp.wav")
lowAmp = os.path.join(DIR, "sounds\\lowAmp.wav")

In [3]:
def loadSound(mainSound, sampleRate=22050):
    global mainArray
    global sr   # sample rate
    mainArray, sr = librosa.load(mainSound, sr=sampleRate)

In [4]:
def drawSinPlot(zoomFaktor):
    duration = 2
    x = np.linspace(0, duration, len(mainArray))
    
    plt.figure(figsize=(18,7))
    plt.ylabel("amplitude", fontsize=15)
    plt.xlabel("time", fontsize=15)
    plt.title(name, fontsize=15)
    plt.plot(x[:zoomFaktor], mainArray[:zoomFaktor])
    plt.show()

In [5]:
def drawIftPlot(zoomFaktor):
    duration = 2
    x = np.linspace(0, duration, len(mainArray))
    
    plt.figure(figsize=(18,7))
    plt.ylabel("amplitude", fontsize=15)
    plt.xlabel("time", fontsize=15)
    plt.title(name, fontsize=15)
    plt.plot(x[:zoomFaktor], mainArray[:zoomFaktor] , "-b", label="Original Signal")
    plt.plot(x[:zoomFaktor], iftSound.real[:zoomFaktor], "-r" ,label='Inverse Fourier-Transformed ')
    plt.legend()
    plt.show()

In [6]:
def drawFtPlot(upperAdjustment, lowerAdjustment):
    ftXAxis= np.linspace(0, sr, len(np.abs(ftSound)))
    
    plt.figure(figsize=(18,7))
    plt.xlabel("frequency", fontsize=15)
    plt.title(name + " Fourier Transformed", fontsize=15)
    plt.plot(ftXAxis[lowerAdjustment:upperAdjustment], np.abs(ftSound)[lowerAdjustment:upperAdjustment])
    plt.show()

In [7]:
def bandpassFilter(freqMin, freqMax):
    """
    inspired from: https://github.com/alessandro-gentilini/opencv_exercises-butterworth/issues/1
    """
    tempFft = ftSound.copy()
    
    frequencies = np.fft.fftfreq(mainArray.shape[0], d=1.0/sr)
    
    # Indizes des unteren endes finden
    boundLow = (np.abs(frequencies - freqMin)).argmin()
    # indizes des oberen endes finden
    boundHigh = (np.abs(frequencies - freqMax)).argmin()

    tempFft[:boundLow] = 0
    tempFft[-boundLow:] = 0
    tempFft[boundHigh:-boundHigh] = 0
    
    transformIfft(tempFft)

## Einführung

Eine Frequenz beschreibt die Zyklen pro Sekunde für ein beliebiges Signal.
Frequenzen werden in Hertz angegeben.

$$
\begin{aligned}
Hz = 1s^{-1}
\end{aligned}
$$

Das menschliche Gehör kann das Spektrum von 20Hz - 20.000Hz noch erfassen.  
Wobei sich jedoch die menschliche Sprache nur von 85Hz - 100Hz strecken kann.  

Im Folgenden werden kurz die zwei Hauptmerkmale von Tönen beschrieben.  

1) Frequenzen: Je höher die Frequenz ist, desto höher ist der Ton. Somit werden mit tieferen Frequenzen tiefere Töne erreicht.

In [8]:
#Ton mit 30 Hz:
lowFreqSound = ipd.Audio(lowFreq)
#Ton mit 800 Hz:
highFreqSound = ipd.Audio(highFreq)
print("Achten Sie auf die Lautstärke! Beginnen Sie mit einer geringen Lautstärke und stellen Sie die Lautstärke von dort aus ein\n")
display(lowFreqSound)
display(highFreqSound)

Achten Sie auf die Lautstärke! Beginnen Sie mit einer geringen Lautstärke und stellen Sie die Lautstärke von dort aus ein



2) Amplitude: Je höher die Amplitude eines Tons ist, desto lauter ist er. Im Gegenschluss ist der Ton leiser je kleiner die Amplitude.

In [9]:
#Ton mit kleiner amplitude:
lowAmpSound = ipd.Audio(lowAmp)
#Ton mit großer amplitude:
highAmpSound = ipd.Audio(highAmp)
print("Achten Sie auf die Lautstärke! Beginnen Sie mit einer geringen Lautstärke und stellen Sie die Lautstärke von dort aus ein\n")
display(lowAmpSound)
display(highAmpSound)

Achten Sie auf die Lautstärke! Beginnen Sie mit einer geringen Lautstärke und stellen Sie die Lautstärke von dort aus ein



## Menü Erzeugung und Signal auswahl

In [10]:
sound = widgets.Dropdown(
       options=["410Hz+420Hz+430Hz", "410Hz+510Hz+610Hz", "Square Signal 100Hz"],
       value="410Hz+510Hz+610Hz",
       description="Sound:")
sound

Dropdown(description='Sound:', index=1, options=('410Hz+420Hz+430Hz', '410Hz+510Hz+610Hz', 'Square Signal 100H…

In [18]:
if sound.value == "410Hz+420Hz+430Hz":
    mainSound = multi400
    global name
    name = "Sinus 410Hz + 420Hz + 430Hz"
elif sound.value == "410Hz+510Hz+610Hz":
    mainSound = multi456
    name = "Sinus 410Hz + 510Hz + 610Hz"
elif sound.value == "Square Signal 100Hz":
    mainSound = square100
    name = "Square Signal 100Hz"
else:
    raise NameError("Chosen sound doesnt match any sounds")

loadSound(mainSound)

print("\nJetzt können Sie sich den Ton anhören, auf dem wir die Fourier-Transformation ausführen werden")
print("Achten Sie auf die Lautstärke! Beginnen Sie mit einer geringen Lautstärke und stellen Sie die Lautstärke von dort aus ein\n")
ipd.Audio(mainSound)


Jetzt können Sie sich den Ton anhören, auf dem wir die Fourier-Transformation ausführen werden
Achten Sie auf die Lautstärke! Beginnen Sie mit einer geringen Lautstärke und stellen Sie die Lautstärke von dort aus ein



Wie man anhand dieses Tones möglicherweise erhören kann handelt es sich hierbei um ein Signal, dass sich aus mehreren Signalen mit verschiedenen Frequenzen zusammensetzt.

Dies basiert auf dem Prinzip der Interferenz. Hierbei wird die Auslenkung der einzelnen Signale zusammenaddiert, wodurch sich die Amplitude im zusammengesetzten Signal vergrößert oder verkleinert.  
Durch die unterschiedliche Amplitudengröße erhält man somit den Ton, der lauter und leiser wird.  

Dies ist in folgendem Bild verdeutlicht. 
<h2><center>Interferenzen Beispiel</center></h2>
<center><img src="pictures/inteferenzen.png"/></center>
<a href=https://userpages.uni-koblenz.de/~unikorn/lehre/gdrn/ws20/02%20Physikalische%20Schicht%20(leer).pdf>Quelle</a>

## Visualierung der original Daten

In [19]:
slider = interact(drawSinPlot, zoomFaktor=widgets.IntSlider(min=0, max=sr/2, step= 10, value=sr/2, continuous_update=False))
display(slider)

interactive(children=(IntSlider(value=11025, continuous_update=False, description='zoomFaktor', max=11025, ste…

<function __main__.drawSinPlot(zoomFaktor)>

## Ausführung der Fourier-Transformation

Mithilfe der Fourier-Transformation kann ein periodisches zeitdiskretes Signal aus dem Zeitspektrum in das Frequenzspektrum überführt werden. 

Diese Verfahren wird in der Signalverarbeitung oft verwendet, um die einzelnen Frequenzen in einem zusammengesetzten Signal zu identifizieren oder nicht periodische Funktionen durch die Linearkombination aus Sinus und Cosinus-Funktionen (welche verschiedene Frequenzen haben) darzustellen.

Da im praktischen meistens eine Menge diskreter Werte vorliegt wird eine Diskrete Fourier-Transformation durchgeführt.  
Es sind verschiedene Variationen der Fourier-Transformation möglich, die sich im Vorzeichen des Exponenten oder der Normalisierung bemerkbar machen.  
Im Folgenden wird diese Varianten der Fourier-Transformation verwendet.

$$
\begin{aligned}
X[k] = \sum_{n=0}^{N-1} x[n]*e^{-2\pi i \dfrac{mk}{N}} \quad \quad \quad k = 0, .., n-1
\end{aligned}
$$

In [20]:
global ftSound
ftSound = np.fft.fft(mainArray)

In [21]:
print("Aufgrund der Symmetrie wird hier nur die Hälfte der Fourier Transformation gezeigt")
slider = interact(drawFtPlot, upperAdjustment=widgets.IntSlider(min=0, max=int(len(ftSound)/2), step=10, value=int(len(ftSound)/2), continuous_update=False), lowerAdjustment=widgets.IntSlider(min=0, max=int(len(ftSound)/2), step=10, value=0, continuous_update=False))
display(slider)

Aufgrund der Symmetrie wird hier nur die Hälfte der Fourier Transformation gezeigt


interactive(children=(IntSlider(value=22050, continuous_update=False, description='upperAdjustment', max=22050…

<function __main__.drawFtPlot(upperAdjustment, lowerAdjustment)>

<font color="red"><strong><em> Information: Die nächsten Zellen müssen ausgeführt werden, jedoch werden die Funktionen zum Schluss in einem interaktiven Plot verwendet.  
Aus Übersichtlichkeitsgründen wurde die Reihenfolge so gewählt. </em></strong></font>

## Ausführung der inversen Fourier-Transformation

Durch die inverse Fourier-Transformation wird die Rücktransformation vom Frequenzspektrum in das Zeitsprektrum erfüllt.  

In dieser Implementierung wird die Normalisierung hier durchgeführt. (zu sehen an $\dfrac{1}{N}$).  

$$
\begin{aligned}
x[n] = \dfrac{1}{N} \sum_{k=0}^{N-1} X[k]*e^{2\pi i \dfrac{mk}{N}} \quad \quad \quad m = 0, .., n-1
\end{aligned}
$$

In [22]:
def transformIfft(fftData):
    global iftSound
    iftSound = np.fft.ifft(fftData)
    print("    RMSE :" + str(calcRMSE()))
    slider = interact(drawIftPlot, zoomFaktor=widgets.IntSlider(min=0, max=44100, step= 10, value=22050, continuous_update=False))
    display(slider)

## Grad der Abweichung

Zur Bestimmung der Abweichung von dem original zum invers Fourier-Transformierten Signal wird der Root Mean Square Error (RMSE) verwendet.  
$$
\begin{aligned}
RMSE = \sqrt{\dfrac{1}{N} \sum_{i=0}^{N}(y_i - \hat{y_i})^2} \\ \\
N = Anzahl \space an\space verglichenen\space Werten\\
y_i = Tatsächliche\space Daten\\
\hat{y_i} = Berechnete\space Daten (Vorhergesagte\space Daten\space häufig)\\
\end{aligned}
$$

Mithilfe dieses Verfahrens kann eine Beurteilung über die Übereinstimmung eines generierten Signals zu einem vorliegendem Signal getätigt werden.  
Häufige Verwendung des RMSE ist bei der Vorhersage von Daten. Hierbei werden vorhergesagte Daten (Corona Daten, Wetter Daten, etc..) mit tatsächlichen (eingetretenen) Daten verglichen, um die Genauigkeit eines Algorithmus zu beurteilen.  
> Je größer der RMSE ist, desto weiter entfernt sind die verglichenen Werte und somit ist der benutzte Algorithmus schlechter.  
> Je kleiner der RMSE wird, desto besser wird der Algorithmus. Ideal wäre ein Wert von 0, der aber in der Realität fast nie erreicht wird.

In [23]:
def calcRMSE():
    RMSE = sqrt(np.square(np.subtract(iftSound.real, mainArray)).mean())
    return RMSE.real

## Filterung des Signals

Mithilfe eines Bandpasses wird im folgenden ein Frequenzbereich ausgewählt, der mithilfe der Fourier-Transformation in das Zeitspektrum zurücktransformiert wird.  
Bei der Rücktransformation werden nur Frequenzen berücksichtigt, die sich in dem angegebenen Frequenzband befinden (zwischen freqMin und freqMax).  
Frequenzen, die nicht in dem angegebenen Frequenzband liegen, werden nicht berücksichtigt, wodurch Abweichungen zum originalen Signal eintreten können.  
>Anzumerken ist, dass im unteren interaktiven Plot die Grenzen nicht inklusive sind.  
Dies bedeutet, dass, falls das Signal eine Frequenz von 410 Hz hat und freqMin auf 410 Hz eingestellt ist diese Frequenz nicht berücksichtigt wird und somit der RMSE steigt.  Bei einem freqMin von 400 Hz und einem frqMax > als 410 Hz würde die Frequenz im Bandpass liegen. Falls diese die einzige Frequenz im Signal ist, würde der RMSE sehr klein werden.

In [24]:
# Because using the slider with 20000 as max Value we just set it when we want to analyze the rect signal
# In this range you can see all of the different forms of the rect signal
# The value is set to 99 and doesnt need to be changed
if name == "Square Signal 100Hz":
    maxValue = 20000
    value = 99
else:
    maxValue = 1000
    value = 410
sliderFilter = interact(bandpassFilter, freqMin=widgets.IntSlider(min=0, max=maxValue, step= 1, value=value, continuous_update=False), freqMax=widgets.IntSlider(min=0, max=maxValue, step= 1, value=610, continuous_update=False))
display(sliderFilter)

interactive(children=(IntSlider(value=99, continuous_update=False, description='freqMin', max=20000), IntSlide…

<function __main__.bandpassFilter(freqMin, freqMax)>